In [1]:
import requests
import mysql.connector
from mysql.connector import Error
import schedule
import time
 


In [2]:
# Airtable configuration
AIRTABLE_BASE_ID = 'appGnf8ldkBqyxsdX'
AIRTABLE_API_KEY = 'patb92KmkkX08CuWq.3e77882b6467bc2705d5f93a6468e7d74f8276494194126fe99328891d59f46b'
AIRTABLE_URL = f'https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/'

# MySQL configuration
MYSQL_HOST = 'ugandarugbyunion.c5s2s2ki82x4.eu-north-1.rds.amazonaws.com'
MYSQL_DATABASE = 'ugandarugbyunion'
MYSQL_USER = 'admin'
MYSQL_PASSWORD = '2210kpeteR$$'

In [3]:
# Function to fetch records from Airtable
table_name = "players"
def fetch_airtable_records(table_name):
    headers = {
        'Authorization': f'Bearer {AIRTABLE_API_KEY}',
        'Content-Type': 'application/json'
    }
    response = requests.get(f'{AIRTABLE_URL}{table_name}', headers=headers)
    
    if response.status_code == 200:
        return response.json().get('records', [])
    else:
        print(f"Failed to fetch records from Airtable: {response.status_code}, Response: {response.text}")
        return []
records = fetch_airtable_records(table_name)
print(f"✅ {table_name}: {len(records)} records fetched.")

✅ players: 100 records fetched.


In [4]:
# Upsert function for inserting or updating MySQL records
def upsert_record(cursor, table, record):
    fields = record.get('fields', {})  # Fall back to empty dict if fields are not found

    if table == 'players':
        sql = """
        INSERT INTO players (Club, PlayerName, DOB, Status, PlayerImage)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            DOB = VALUES(DOB),
            Status = VALUES(Status),
            PlayerImage = VALUES(PlayerImage);
        """

        # Get values from fields, replacing None with 0 for Club if necessary
        club_field = fields.get('Club')
        if club_field is None:
            club_field = 0  # Default to 0 if Club is missing

        values = (
            club_field,
            fields.get('PlayerName'),
            fields.get('DOB'),  # Ensure date is in the correct format
            fields.get('Status'), 
            fields.get('PlayerImage')
        )

        print(f"Executing SQL for players: {sql} with values: {values}")  # Logging
        cursor.execute(sql, values)
    else:
        print(f"⚠️ Table '{table}' not handled.")
        return

    try:
        cursor.execute(sql, values)
    except Error as err:
        print(f"Error executing SQL for table {table}: {err}")

In [5]:
# Main function to synchronize Airtable with MySQL
def sync_airtable_to_mysql():
    try:
        connection = mysql.connector.connect(
            host=MYSQL_HOST,
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD
        )
        
        if connection.is_connected():
            cursor = connection.cursor()

            # Fetch current records from MySQL for the players table
            cursor.execute("SELECT Club, PlayerName FROM players;")
            existing_records = { (row[0], row[1]) for row in cursor.fetchall() }  # Set of tuples

            # Fetch records from Airtable specifically for players
            records = fetch_airtable_records('players')  # Specify table name
            
            # Prepare to track identifiers fetched from Airtable
            fetched_identifiers = set()

            for record in records:
                identifier = (record['fields'].get('Club'), record['fields'].get('PlayerName'))  # Create tuple for identifiers
                upsert_record(cursor, 'players', record)

                # Track fetched identifiers (ensure they are tuples)
                fetched_identifiers.add(identifier)

            # Deletions: Identify records in MySQL not found in Airtable
            for identifier in existing_records:
                if identifier not in fetched_identifiers:
                    # Create delete SQL statement
                    delete_sql = "DELETE FROM players WHERE Club = %s AND PlayerName = %s;"
                    cursor.execute(delete_sql, identifier)

            connection.commit()  # Commit all changes
            print("Data synchronized successfully, including deletions.")

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Schedule the sync function to run every minute
schedule.every(1).minutes.do(sync_airtable_to_mysql)

# Keep the script running to maintain the scheduling
while True:
    schedule.run_pending()
    time.sleep(1)  # Sleep to prevent high CPU usage

Executing SQL for players: 
        INSERT INTO players (Club, PlayerName, DOB, Status, PlayerImage)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            DOB = VALUES(DOB),
            Status = VALUES(Status),
            PlayerImage = VALUES(PlayerImage);
         with values: ('Kobs ', 'Adnan Mutebi', '1992-01-04', 'Active', '0')
Executing SQL for players: 
        INSERT INTO players (Club, PlayerName, DOB, Status, PlayerImage)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            DOB = VALUES(DOB),
            Status = VALUES(Status),
            PlayerImage = VALUES(PlayerImage);
         with values: ('Pirates ', 'Raphael Anyama', '1996-06-08', 'Active', '0')
Executing SQL for players: 
        INSERT INTO players (Club, PlayerName, DOB, Status, PlayerImage)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            DOB = VALUES(DOB),
            Status = VALUES(Status),
            PlayerImage = VALUES(

UnboundLocalError: cannot access local variable 'connection' where it is not associated with a value